# NOAA
- request example: https://www.ncei.noaa.gov/access/services/data/v1?dataset=local-climatological-data&dataTypes=HourlyWindSpeed&dataTypes=HourlyDryBulbTemperature&stations=72530094846&startDate=2018-05-01&endDate=2018-05-31
- list of stationsids: https://www.itl.nist.gov/div898/winds/asos-wx/WBAN-MSC.TXT
- Other Notes: https://www.ncei.noaa.gov/access/services/support/v3/datasets.json

In [6]:
#needed to make web requests
import requests

#store the data we get as a dataframe
import pandas as pd

#convert the response as a strcuctured json
import json

#mathematical operations on lists
import numpy as np

#parse the datetimes we get from NOAA
from datetime import datetime

import urllib

pd.set_option('display.max_columns', None)

pd.set_option('display.max_rows', None)

In [71]:
# sites
stations_link = 'https://www.itl.nist.gov/div898/winds/asos-wx/WBAN-MSC.TXT'
df = pd.read_csv(stations_link, skiprows=5, sep='\s+',on_bad_lines='skip')
df.fillna('',inplace=True)
df['NAME']  = df['WBAN'] + ' ' +df['NAME']
df.drop(columns='WBAN',inplace=True)
df.rename(columns={'AWSMSC':'WBAN'},inplace=True)
df['AWSMSC'] = df.index
df.reset_index(drop=True, inplace=True)
#df['AWSMSC_WBAN'] = df.index) + str(df['AWSMSC_WBAN'])


In [80]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 889 entries, 0 to 888
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   WBAN    889 non-null    int64 
 1   NAME    889 non-null    object
 2   AWSMSC  889 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 21.0+ KB


In [79]:
df.AWSMSC.astype(str) +  df.WBAN.astype(str)

0       3068015044
1       3135015039
2       3171015002
3       3470135055
4       3562315033
5       3577335046
6       3583335034
7       3595335051
8       3596335048
9       3655315037
10      3688335050
11      3743515040
12      3797035047
13      4018016201
14      4210117501
15      4218316505
16      4220016502
17      4230016507
18      4231016504
19      4250016506
20      4260016415
21      4261016404
22      4270016405
23      4280316402
24      4330017203
25      7033034066
26      7038034061
27      7088134064
28      7147034045
29      7149034022
30      7165334067
31      7179034051
32      7186034062
33      7217014009
34      7260034059
35      7354034048
36      7480034024
37      7647034018
38      7650034012
39      7790034017
40      8160614010
41      8221014007
42      8224014012
43      8227014011
44      8449013025
45      8505013202
46      8509013201
47     10136035002
48     10224035001
49     10246035108
50     10382035110
51     10384035104
52     10439

numpy.int64